In [1]:
#| default_exp workflow

In [4]:
#| export

from chewc.structs import *
from chewc.pheno import *
from chewc.select import *
from chewc.cross import *
import jax
import jax.numpy as jnp
from jax import lax
from functools import partial
from typing import Tuple

print("This document represents a template for a simple breeding simulation for selection. It is meant to be a guide to developing my complex or flexible breeding designs.")


@partial(jax.jit, static_argnames=("n_select", "n_offspring", "max_crossovers"))
def selection_step(
    carry: BreedingState,
    _, # Placeholder for lax.scan's iteration number
    trait: Trait,
    genetic_map: GeneticMap,
    heritabilities: jnp.ndarray,
    n_select: int,
    n_offspring: int,
    max_crossovers: int
) -> Tuple[BreedingState, jnp.ndarray]:
    """
    Executes one full cycle of selection and breeding.
    This function is designed to be the body of a lax.scan loop.
    """
    key, pheno_key, mating_key, cross_key = jax.random.split(carry.key, 4)
    current_pop = carry.population

    # 1. Evaluate the population
    phenotypes, tbvs = calculate_phenotypes(
        key=pheno_key, population=current_pop, trait=trait, heritability=heritabilities
    )
    mean_tbv = jnp.mean(tbvs[:, 0]) # Track genetic gain for trait 1

    # 2. Select top parents
    selected_parents = select_top_k(current_pop, phenotypes[:, 0], k=n_select)

    # 3. Generate a random mating plan
    pairings = random_mating(mating_key, n_parents=n_select, n_crosses=n_offspring)
    mother_indices, father_indices = pairings[:, 0], pairings[:, 1]

    # 4. Create the next generation
    mothers_geno = selected_parents.geno[mother_indices]
    fathers_geno = selected_parents.geno[father_indices]
    mothers_ibd = selected_parents.ibd[mother_indices]
    fathers_ibd = selected_parents.ibd[father_indices]

    # Vectorize the crossing operation over all pairs
    vmapped_cross = jax.vmap(
        cross_pair, in_axes=(0, 0, 0, 0, 0, None, None)
    )
    offspring_keys = jax.random.split(cross_key, n_offspring)
    offspring_geno, offspring_ibd = vmapped_cross(
        offspring_keys, mothers_geno, fathers_geno, mothers_ibd, fathers_ibd, genetic_map, max_crossovers
    )

    # 5. Form the new population and update the state
    new_generation = carry.generation + 1
    new_ids = jnp.arange(n_offspring, dtype=jnp.int32) + carry.next_id
    new_meta = jnp.stack(
        [
            new_ids,
            selected_parents.meta[mother_indices, 0],
            selected_parents.meta[father_indices, 0],
            jnp.full((n_offspring,), new_generation, dtype=jnp.int32),
        ],
        axis=-1,
    )
    new_population = Population(geno=offspring_geno, ibd=offspring_ibd, meta=new_meta)

    next_state = BreedingState(
        population=new_population,
        key=key,
        generation=new_generation,
        next_id=carry.next_id + n_offspring
    )

    # The scan function requires a `(carry, output)` return signature.
    # `carry` is the state for the next iteration.
    # `output` is the data we want to collect at each step.
    return next_state, mean_tbv


def run_simulation_cycles(
    initial_state: BreedingState,
    trait: Trait,
    genetic_map: GeneticMap,
    heritabilities: jnp.ndarray,
    n_cycles: int,
    n_select: int,
    n_offspring: int,
    max_crossovers: int
):
    """
    Runs the entire multi-cycle simulation using lax.scan for maximum performance.
    """
    # `lax.scan` requires the body function to be a callable that takes
    # (carry, x) and returns (new_carry, y). We use a lambda to fix the
    # static arguments and data that doesn't change over the loop.
    scan_fn = lambda carry, _: selection_step(
        carry,
        _,
        trait=trait,
        genetic_map=genetic_map,
        heritabilities=heritabilities,
        n_select=n_select,
        n_offspring=n_offspring,
        max_crossovers=max_crossovers
    )

    # Run the scan. The `None` is a placeholder for the `xs` array,
    # as we only care about the number of iterations (`length`).
    final_state, tbv_history = lax.scan(
        scan_fn, initial_state, None, length=n_cycles
    )

    return final_state, tbv_history


# ----------------------------------------------------
# Main execution script
# ----------------------------------------------------

if __name__ == "__main__":
    print("--- Running 10-Cycle Phenotypic Selection Experiment ---")

    # --- Simulation Parameters ---
    N_FOUNDERS = 100
    N_SELECT = 20
    N_OFFSPRING = 200 # Population size is kept constant after the first cycle
    N_CYCLES = 20

    N_CHR, N_LOCI = 5, 1000
    MAX_CROSSOVERS = 10
    SEED = 42

    key = jax.random.PRNGKey(SEED)

    # --- Setup ---
    key, pop_key, trait_key = jax.random.split(key, 3)
    print("\nStep 1: Initializing founder population...")
    founder_pop, genetic_map = quick_haplo(
        key=pop_key, n_ind=N_FOUNDERS, n_chr=N_CHR, seg_sites=N_LOCI,
        inbred=False, chr_length=1.0
    )

    print("Step 2: Initializing trait architecture...")
    trait_architecture = add_trait(
        key=trait_key, founder_pop=founder_pop, n_qtl_per_chr=50,
        mean=jnp.array([0.0]), var_a=jnp.array([1.0]), var_d=jnp.array([0.0]),
        sigma=jnp.array([[10.0]])
    )
    heritabilities = jnp.array([0.9])

    # --- Initial State ---
    initial_state = BreedingState(
        population=founder_pop,
        key=key,
        generation=0,
        next_id=N_FOUNDERS # Next available individual ID
    )
    print(f"  - Founder population size: {N_FOUNDERS}")
    print(f"  - Selection: Top {N_SELECT} individuals")
    print(f"  - Offspring per cycle: {N_OFFSPRING}")

    # --- Burn-in Cycle ---
    print("\nStep 3: Running burn-in cycle to stabilize population size...")
    
    # Calculate initial TBV for generation 0
    key, initial_pheno_key = jax.random.split(initial_state.key)
    _, initial_tbvs = calculate_phenotypes(
        key=initial_pheno_key, population=initial_state.population,
        trait=trait_architecture, heritability=heritabilities
    )
    initial_mean_tbv = jnp.mean(initial_tbvs[:, 0])

    # Run the first selection step manually
    burn_in_state, first_cycle_tbv = selection_step(
        carry=initial_state,
        _ = 0, # Placeholder for iteration number
        trait=trait_architecture,
        genetic_map=genetic_map,
        heritabilities=heritabilities,
        n_select=N_SELECT,
        n_offspring=N_OFFSPRING,
        max_crossovers=MAX_CROSSOVERS,
    )

    # --- Run Simulation ---
    print(f"\nStep 4: Running remaining {N_CYCLES - 1} selection cycles (JIT compiling...)\n")
    
    # Run the rest of the simulation using lax.scan
    final_state, tbv_history = run_simulation_cycles(
        initial_state=burn_in_state,
        trait=trait_architecture,
        genetic_map=genetic_map,
        heritabilities=heritabilities,
        n_cycles=N_CYCLES - 1,
        n_select=N_SELECT,
        n_offspring=N_OFFSPRING,
        max_crossovers=MAX_CROSSOVERS,
    )

    # --- Report Results ---
    print("--- Results ---")
    print(f"Generation 00 Mean TBV: {initial_mean_tbv:.3f}")
    print(f"Generation 01 Mean TBV: {first_cycle_tbv:.3f} (Gain: {first_cycle_tbv - initial_mean_tbv:+.3f})")

    # Combine initial TBV, burn-in TBV, and the history from the scan
    full_history = jnp.concatenate([jnp.array([initial_mean_tbv, first_cycle_tbv]), tbv_history])
    for i, tbv in enumerate(tbv_history, 2):
        gain = tbv - full_history[i-1]
        print(f"Generation {i:02d} Mean TBV: {tbv:.3f} (Gain: {gain:+.3f})")

    total_gain = tbv_history[-1] - initial_mean_tbv
    print(f"\n--- Total Genetic Gain over {N_CYCLES} generations: {total_gain:+.3f} ---")

This document represents a template for a simple breeding simulation for selection. It is meant to be a guide to developing my complex or flexible breeding designs.
--- Running 10-Cycle Phenotypic Selection Experiment ---

Step 1: Initializing founder population...
Step 2: Initializing trait architecture...
  - Founder population size: 100
  - Selection: Top 20 individuals
  - Offspring per cycle: 200

Step 3: Running burn-in cycle to stabilize population size...

Step 4: Running remaining 19 selection cycles (JIT compiling...)

--- Results ---
Generation 00 Mean TBV: 0.816
Generation 01 Mean TBV: 0.816 (Gain: +0.000)
Generation 02 Mean TBV: 2.206 (Gain: +1.390)
Generation 03 Mean TBV: 3.775 (Gain: +1.569)
Generation 04 Mean TBV: 5.289 (Gain: +1.514)
Generation 05 Mean TBV: 6.590 (Gain: +1.301)
Generation 06 Mean TBV: 7.983 (Gain: +1.394)
Generation 07 Mean TBV: 9.112 (Gain: +1.129)
Generation 08 Mean TBV: 10.077 (Gain: +0.965)
Generation 09 Mean TBV: 10.777 (Gain: +0.700)
Generation 1

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()